# MNL demo

Sam Maurer | Python 3.6

In [1]:
import os; os.chdir('../')

In [2]:
import numpy as np
import pandas as pd
from collections import OrderedDict

In [3]:
from urbansim_templates import modelmanager as mm
from urbansim_templates.models import BinaryLogitStep, OLSRegressionStep, SmallMultinomialLogitStep
import orca

/Users/maurer/anaconda3/envs/ual-model/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [4]:
# Load any script-based Orca registrations
from scripts import datasources
from scripts import models

In [5]:
for table_name in orca.list_tables():
    print(table_name.upper())
    print(orca.get_table(table_name).to_frame().columns.tolist())
    print()

HOUSEHOLDS
['household_id', 'taz', 'serialno', 'puma5', 'income', 'persons', 'hht', 'unittype', 'noc', 'bldgsz', 'tenure', 'vehicl', 'hinccat1', 'hinccat2', 'hhagecat', 'hsizecat', 'hfamily', 'hunittype', 'hnoccat', 'hwrkrcat', 'h0004', 'h0511', 'h1215', 'h1617', 'h1824', 'h2534', 'h3549', 'h5064', 'h6579', 'h80up', 'hworkers', 'hwork_f', 'hwork_p', 'huniv', 'hnwork', 'hretire', 'hpresch', 'hschpred', 'hschdriv', 'htypdwel', 'hownrent', 'hadnwst', 'hadwpst', 'hadkids', 'bucketbin', 'originalpuma', 'hmultiunit', 'building_id']

BUILDINGS
['building_id', 'parcel_id', 'development_type_id', 'improvement_value', 'residential_units', 'residential_sqft', 'sqft_per_unit', 'non_residential_sqft', 'building_sqft', 'nonres_rent_per_sqft', 'res_price_per_sqft', 'stories', 'year_built', 'redfin_sale_price', 'redfin_sale_year', 'redfin_home_type', 'costar_property_type', 'costar_rent', 'building_type_id']



### Small MNL

In [6]:
orca.get_table('households').to_frame(['tenure']).tenure.unique()

# i think 1=own w/mortgage, 2=own free&clear, 3=rent, 4=no payment

array([3, 2, 4, 1])

In [7]:
m = SmallMultinomialLogitStep()
m.tables = ['households']
m.choice_column = 'tenure'
m.filters = ['household_id % 1000 < 1',
             'tenure < 4']

m.model_expression = OrderedDict([
    ('intercept', [2,3]),        
    ('income', [1,3]),
    ('persons', [1,3])])

m.name = 'small-mnl-test'
m.tags = ['sam', 'testing']
m.out_column = 'taz'

In [8]:
len(m._get_data())

2575

In [9]:
%%time
m.fit()

Log-likelihood at zero: -2,828.9266
Initial Log-likelihood: -2,828.9266
Estimation Time for Point Estimation: 0.05 seconds.
Final log-likelihood: -2,419.4331


/Users/maurer/anaconda3/envs/ual-model/lib/python3.6/site-packages/scipy/optimize/_minimize.py:420: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  RuntimeWarning)


                     Multinomial Logit Model Regression Results                    
Dep. Variable:                     _chosen   No. Observations:                2,575
Model:             Multinomial Logit Model   Df Residuals:                    2,569
Method:                                MLE   Df Model:                            6
Date:                     Fri, 23 Mar 2018   Pseudo R-squ.:                   0.145
Time:                             21:35:44   Pseudo R-bar-squ.:               0.143
AIC:                             4,850.866   Log-Likelihood:             -2,419.433
BIC:                             4,885.988   LL-Null:                    -2,828.927
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
intercept_2     0.0230      0.132      0.174      0.862      -0.236       0.282
intercept_3     1.0465      0.100     10.498      0.000       0.851       1.242
income_1

In [10]:
orca.get_table('households').to_frame().taz.describe()

count    2.608019e+06
mean     7.526463e+02
std      4.301403e+02
min      1.000000e+00
25%      3.760000e+02
50%      7.630000e+02
75%      1.146000e+03
max      1.454000e+03
Name: taz, dtype: float64

In [11]:
%%time
m.run()

CPU times: user 10.1 s, sys: 3.18 s, total: 13.3 s
Wall time: 13.5 s


In [12]:
orca.get_table('households').to_frame().taz.describe()

count    2.608019e+06
mean     1.949268e+00
std      9.233045e-01
min      1.000000e+00
25%      1.000000e+00
50%      2.000000e+00
75%      3.000000e+00
max      3.000000e+00
Name: taz, dtype: float64

In [13]:
m.register()

Model saved to configs/small-mnl-test.pkl


In [14]:
mm.get_step('small-mnl-test').run()

In [15]:
mm.get_step('small-mnl-test').fit()

Log-likelihood at zero: -2,828.9266
Initial Log-likelihood: -2,828.9266
Estimation Time for Point Estimation: 0.05 seconds.
Final log-likelihood: -2,419.4331


/Users/maurer/anaconda3/envs/ual-model/lib/python3.6/site-packages/scipy/optimize/_minimize.py:420: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  RuntimeWarning)


                     Multinomial Logit Model Regression Results                    
Dep. Variable:                     _chosen   No. Observations:                2,575
Model:             Multinomial Logit Model   Df Residuals:                    2,569
Method:                                MLE   Df Model:                            6
Date:                     Fri, 23 Mar 2018   Pseudo R-squ.:                   0.145
Time:                             21:36:15   Pseudo R-bar-squ.:               0.143
AIC:                             4,850.866   Log-Likelihood:             -2,419.433
BIC:                             4,885.988   LL-Null:                    -2,828.927
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
intercept_2     0.0230      0.132      0.174      0.862      -0.236       0.282
intercept_3     1.0465      0.100     10.498      0.000       0.851       1.242
income_1

In [16]:
print "break"

SyntaxError: Missing parentheses in call to 'print'. Did you mean print("break")? (<ipython-input-16-5db00b99dd6a>, line 1)

### Work for list of alternative id's

In [ ]:
l = []
for k, v in m.model_expression.items():
    if isinstance(v, list):
        l += v
    else:
        l += [v]
        
l = np.unique(l)
print(l)

In [ ]:
[k for (k,v) in m.model_expression.items()]

In [ ]:
[v for (k,v) in m.model_expression.items()]

### Work for Monte Carlo

In [ ]:
df = m._get_data('predict')
print(len(df))

In [ ]:
long_df = m._to_long(df, 'predict')
print(len(long_df))

In [ ]:
probs = m.model.predict(long_df)
print(len(probs))
print(probs[:10])

In [ ]:
probs2 = probs.reshape((len(df), 4))
print(len(probs2))
print(probs2[:2])

In [ ]:
cumprobs = probs2.cumsum(axis=1)
print(cumprobs[:2])

In [ ]:
rands = np.random.random(len(df))
diff = np.subtract(cumprobs.transpose(), rands).transpose()
print(diff[:2])

In [ ]:
tk = (diff + 1.0).astype('i4')
print(tk[:2])

In [ ]:
choices = np.argmax((diff + 1.0).astype('i4'), axis=1)
print(len(choices))
print(choices[:5])

In [ ]:
choice_indexes = np.array([1,1,0,0,0])
nums = np.arange(10)
nums.take(choice_indexes + np.arange(5)*2)

In [ ]:
df = m._get_data()
print(df.columns.tolist())

In [ ]:
alts = df['tenure'].sort_values().unique().tolist()

#alts_df = pd.DataFrame({'_alt_id': alts}, index=alts)
#alts_df.index = alts

In [ ]:
alts

In [ ]:
obs = df.index.sort_values().unique()

obs_df = pd.DataFrame({'_obs_id': obs}, index=obs)

print(len(obs_df))

In [ ]:
obs = df.index.sort_values().unique().tolist()

In [ ]:
obs_prod, alts_prod = pd.core.reshape.util.cartesian_product([obs, alts])

long_df = pd.DataFrame({'_obs_id': obs_prod, '_alts_id': alts_prod})
print(len(long_df))

In [ ]:
print(long_df.head())

In [ ]:
long_df = long_df.merge(df, left_on='_obs_id', right_index=True)
print(df.columns.tolist())

In [ ]:
%%time
long_df['_chosen'] = 0
long_df.loc[long_df._alts_id == long_df['tenure'], '_chosen'] = 1

In [ ]:
print(long_df.head(3))

In [ ]:
d = OrderedDict([('a', 'b')])
d['a'] = 'b'
d['c'] = 'd'
print(d)

In [ ]:
[k[0] for k in list(d.items()) if k[0] is not 'c']